In [1]:
import dagshub
dagshub.init(repo_owner='sahilbhardwaj23', repo_name='Youtube-comment-Sentiment-Analysis', mlflow=True)

Accessing as sahilbhardwaj23

Initialized MLflow to track repo "sahilbhardwaj23/Youtube-comment-Sentiment-Analysis"

Repository sahilbhardwaj23/Youtube-comment-Sentiment-Analysis initialized!

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow")

In [3]:
from mlflow.tracking import MlflowClient
import mlflow

In [4]:
def set_or_create_experiment(experiment_name):
    client = MlflowClient()
    experiment = client.get_experiment_by_name(experiment_name)

    # Check if the experiment exists and is active
    if experiment:
        if experiment.lifecycle_stage == "active":
            print(f"Using existing experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            mlflow.set_experiment(experiment_name)
        else:
            # Restore if it's deleted
            print(f"Restoring deleted experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            client.restore_experiment(experiment.experiment_id)
            mlflow.set_experiment(experiment_name)
    else:
        # Create a new experiment if it doesn't exist
        print(f"Creating new experiment '{experiment_name}'")
        mlflow.set_experiment(experiment_name)



In [5]:
# Set or create an experiment
set_or_create_experiment("Exp 5 - ML Algos with HP Tuning")

Creating new experiment 'Exp 5 - ML Algos with HP Tuning'


2024/11/02 20:44:56 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


In [6]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

e:\Projects\Youtube Comment Analysis\Youtube commnent Plugin\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [8]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2024-11-02 20:54:00,187] A new study created in memory with name: no-name-ebc9bfac-ea8f-4217-937d-2152936afebe
[I 2024-11-02 20:55:49,128] Trial 0 finished with value: 0.5450702304650211 and parameters: {'n_estimators': 176, 'learning_rate': 0.000124538416078991, 'max_depth': 5}. Best is trial 0 with value: 0.5450702304650211.
[I 2024-11-02 21:00:23,266] Trial 1 finished with value: 0.6258011727805809 and parameters: {'n_estimators': 188, 'learning_rate': 0.0030909237126214716, 'max_depth': 8}. Best is trial 1 with value: 0.6258011727805809.
[I 2024-11-02 21:09:16,471] Trial 2 finished with value: 0.6904404745670257 and parameters: {'n_estimators': 244, 'learning_rate': 0.010142984938741446, 'max_depth': 10}. Best is trial 2 with value: 0.6904404745670257.
[I 2024-11-02 21:10:43,976] Trial 3 finished with value: 0.5589799536342561 and parameters: {'n_estimators': 215, 'learning_rate': 0.0016081715309307246, 'max_depth': 4}. Best is trial 2 with value: 0.6904404745670257.
[I 2024-11-